In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
# import torch.nn.utils.prune as prune
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch import Tensor
from torch.nn.parameter import Parameter

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import math
import warnings

In [34]:
BW=6
BX=6
VBLMAX = 0.8 
T0 = 100e-12
kn = 220e-6
Vt = 0.4
alpha = 1.8
CBL = 270e-15
VWL = 0.9
Icell = kn*np.power(VWL-Vt,alpha) # Ideal cell current of the discharge path
delta_VBL_LSB = T0*Icell/CBL #The voltage difference on VBL created by the LSB
kclip = VBLMAX/delta_VBL_LSB 
#kclip = 10000+VBLMAX/delta_VBL_LSB
sigma_Vt = 23.8e-3
sigma_D = alpha*sigma_Vt/(VWL-Vt)

def quantizeWeight(W,BW):
    W = torch.min(W,(1.0-(2**(-(BW-1.0))))*torch.ones_like(W))
    Wbs = []
    Wbi = torch.lt(W,torch.zeros_like(W)).float()
    Wbs.append(Wbi)
    W = (W + Wbi)
    for i in range(BW-1):
        Wbi = torch.ge(W,0.5*torch.ones_like(W)).float()
        Wbs.append(Wbi)
        W = 2.0*W - Wbi
    carry = torch.ge(W,0.5*torch.ones_like(W)).float()
    for i in range(BW):#-1):
        j = BW-1-i
        Wbs[j] = Wbs[j]+carry
        carry = torch.gt(Wbs[j],1.5*torch.ones_like(Wbs[j])).float()
        Wbs[j] = Wbs[j]*torch.ne(Wbs[j],2.0*torch.ones_like(Wbs[j]))
    return Wbs

def reconstructWeight(Wbs,BW):
    W = torch.zeros_like(Wbs[0])
    for j in range(BW):
        multiplier = (0.5)**j
        if (j == 0):
            multiplier = -1.0
        W += Wbs[j] * multiplier
    return W

class DIMAConv2d(nn.Conv2d):
    def __init__(
        self,
        sigma_D = 0,
        layer_index = 0,
        *kargs,
        **kwargs
    ):
#         self.groups = ((self.kernel_size[0]*self.kernel_size[1]*self.in_channels) // 256) + 1
        super(DIMAConv2d, self).__init__(*kargs,**kwargs)
#         self.groups = ((self.kernel_size[0]*self.kernel_size[1]*self.in_channels) // 256) + 1
#         self.output_h = floor( ((h_w[0] + (2 * pad) - ( dilation * (kernel_size[0] - 1) ) - 1 )/ stride) + 1)
#         self.output_w = floor( ((h_w[1] + (2 * pad) - ( dilation * (kernel_size[1] - 1) ) - 1 )/ stride) + 1)
        self.layer_index = layer_index
        self.noise = np.random.normal(0,sigma_D,(BW,self.weight.size()[0],self.weight.size()[1]
                                                 ,self.weight.size()[2],self.weight.size()[3]))
        
    def quantize_activations(self,input):
        if(self.layer_index != 0):
            input = torch.clamp(input,0,6) / 6
            input = 6 * torch.min(self.round_f(input*(2**BX))*(2**(-BX)) ,(1.0-(2**(-BX)))*torch.ones_like(input))
        else:
            input = torch.min(self.round_f(input*(2**BX))*(2**(-BX)) ,(1.0-(2**(-BX)))*torch.ones_like(input))
        return input
    
    def quantize_outputs(self,output):
        output = torch.clamp(output,-6,6)
        output = torch.min(self.round_f((output/6)*(2**(BW-1.0)))*(2.0**(1.0-BW)),(1.0-(2.0**(1.0-BW)))*torch.ones_like(output))
        output = output * 6
        return output 
               
    def round_f(self, x): #rounds a number to the nearest integer with STE for gradients
        x_r = torch.round(x)
        x_g = x
        return (x_r - x_g).detach() + x_g
    
    def quantize_weights(self):
        weight_q = quantizeWeight(self.weight.data,BW)
        for b in range(BW-1):
            weight_q[b+1] = weight_q[b+1]*(1+self.noise[b])
        weight = reconstructWeight(weight_q,BW)
        Wmax = kclip*np.power(2.0,-(BW-1))
        weight = torch.clamp(weight,-Wmax,Wmax)
        return (weight - self.weight).detach() + self.weight
        
    def _conv_forward(self, input, weight):
        if self.padding_mode != 'zeros':
            return F.conv2d(F.pad(input, self._reversed_padding_repeated_twice, mode=self.padding_mode),
                            weight, self.bias, self.stride,
                            _pair(0), self.dilation, self.groups)
        return F.conv2d(input, weight, self.bias, self.stride,
                        self.padding, self.dilation, self.groups)

    def forward(self, input: Tensor) -> Tensor:
        weight = self.quantize_weights()
        input = self.quantize_activations(input)
        output_h = int((((input.size()[2] + (2 * self.padding[0]) - 
                            ( self.dilation[0] * (self.kernel_size[0] - 1) ) - 1 )/ self.stride[0]) + 1) // 1)
        output_w = int((((input.size()[3] + (2 * self.padding[1]) - 
                            ( self.dilation[1] * (self.kernel_size[1] - 1) ) - 1 )/ self.stride[1]) + 1) // 1)
        
        if(self.kernel_size[0]*self.kernel_size[1]*self.in_channels > 256):
            print(1)
            weights = []
            inputs = []
            val = ((self.kernel_size[0]*self.kernel_size[1]*self.in_channels) // 256) + 1
            coeff = self.in_channels // val
            for i in range (val):
                if(i != val-1):
                    temp_weight = torch.zeros_like(weight)
                    temp_input = torch.zeros_like(input)
                    temp_weight[:,i*coeff:(i+1)*coeff,:,:] = weight[:,i*coeff:(i+1)*coeff,:,:]
                    weights.append(temp_weight)
                    temp_input[:,i*coeff:(i+1)*coeff,:,:] = input[:,i*coeff:(i+1)*coeff,:,:]
                    inputs.append(temp_input)
                else:
                    temp_weight = torch.zeros_like(weight)
                    temp_input = torch.zeros_like(input)
                    temp_weight[:,i*coeff:,:,:] = weight[:,i*coeff:,:,:]
                    weights.append(temp_weight)
                    temp_input[:,i*coeff:,:,:] = input[:,i*coeff:,:,:]
                    inputs.append(temp_input)
            output = torch.zeros((input.size()[0],self.out_channels,output_h,output_w))
            for i in range (val):
                out = self._conv_forward(inputs[i],weights[i])
                output += self.quantize_outputs(out)  
#                 output += out
            output_ = self._conv_forward(input, weight)
            print(output_ - output)
#             output_ = self.quantize_outputs(output_)

#         self.groups = ((self.kernel_size[0]*self.kernel_size[1]*self.in_channels) // 256) + 1
#         print(self.groups)
#         if(self.groups != 1):
#             weight = weight.reshape(self.groups,self.out_channels//self.groups, self.in_channels, weight.size()[2], weight.size()[3])
#         output =  self._conv_forward(input, weight)
#         output = self.quantize_outputs(output)
#         print(output.size())
#         if(self.groups != 1):
#             output = torch.sum(output, 2)
        else:
            output = self._conv_forward(input, weight)
            output = self.quantize_outputs(output)
        return output

In [2]:
class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()
        
        self.feature_extractor = nn.Sequential(            
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1,bias=False),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1,bias=False),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1,bias=False),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84,bias=False),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes,bias=False),
        )


    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        return logits, probs
    
# torch.manual_seed(RANDOM_SEED)

# model = LeNet5(N_CLASSES).to(DEVICE)
# optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# criterion = nn.CrossEntropyLoss()
    
# model, optimizer, _ = training_loop(model, criterion, optimizer, train_loader, valid_loader, N_EPOCHS, DEVICE)

def get_datasets(*args, **kwargs):
    transform = transforms.Compose(
        [
            transforms.Resize((32, 32)),
            transforms.ToTensor()
#             transforms.Normalize((0.1307,), (0.3081,))
        ]
    )

    trainset = torchvision.datasets.MNIST(train=True, transform=transform, *args, **kwargs)
    testset = torchvision.datasets.MNIST(train=False, transform=transform, *args, **kwargs)
    return trainset, testset

def get_dataloaders(trainset, testset, batch_size=100, num_worker=4):
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_worker)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_worker)

    return trainloader, testloader

LeNET = LeNet5(10)
criterion = nn.CrossEntropyLoss()
lr_ = 0.002
optimizer = torch.optim.SGD(LeNET.parameters(), lr=lr_)


trainset, testset = get_datasets(root='./data', download=True)
trainloader, testloader = get_dataloaders(trainset, testset, batch_size=100, num_worker=16)
loss_ = np.zeros(15)
gradient = np.zeros((4,15))

for epoch in range(20):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()
        outputs, _ = LeNET(inputs)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()
        running_loss += loss.item()

    
print('Finished Training')

PATH = './mnist_LeNET.pth'
torch.save(model.state_dict(), PATH)

Finished Training


NameError: name 'model' is not defined

In [4]:
LeNET.load_state_dict(torch.load(PATH), strict=False)

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs, _ = LeNET(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 98 %


In [35]:
class LeNet5_DIMA(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5_DIMA, self).__init__()
        
        self.feature_extractor = nn.Sequential(            
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1,bias=False),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1,bias=False),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            DIMAConv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1,bias=False, sigma_D = sigma_D, layer_index = 2),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84,bias=False),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes,bias=False),
        )


    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        return logits, probs

LeNET_DIMA = LeNet5_DIMA(10)
LeNET_DIMA.load_state_dict(torch.load(PATH), strict=False)

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs, _ = LeNET_DIMA(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

1
tensor([[[[ 4.3777e-02]],

         [[-7.6263e-02]],

         [[-1.8691e-02]],

         ...,

         [[-5.7187e-02]],

         [[-3.1710e-02]],

         [[-6.7247e-02]]],


        [[[ 8.3453e-02]],

         [[-9.4068e-02]],

         [[ 1.3244e-01]],

         ...,

         [[-6.5265e-02]],

         [[-1.1063e-02]],

         [[-5.1800e-01]]],


        [[[ 1.3167e-01]],

         [[-4.7019e-02]],

         [[ 1.5836e-02]],

         ...,

         [[ 4.1977e-02]],

         [[-7.8005e-02]],

         [[ 4.2551e-02]]],


        ...,


        [[[-1.4052e-01]],

         [[ 1.3060e-01]],

         [[ 1.2331e-01]],

         ...,

         [[-2.7090e-02]],

         [[-2.5522e-02]],

         [[ 1.2278e-02]]],


        [[[ 1.4842e+00]],

         [[ 7.2918e-02]],

         [[ 5.3406e-05]],

         ...,

         [[ 8.5252e-02]],

         [[ 1.1178e-01]],

         [[-2.8179e+00]]],


        [[[ 4.9670e-01]],

         [[ 5.5733e-02]],

         [[-7.4118e-02]],

       

1
tensor([[[[-0.0787]],

         [[-0.0756]],

         [[ 0.0533]],

         ...,

         [[-0.0335]],

         [[-0.0282]],

         [[-0.0769]]],


        [[[ 1.9202]],

         [[-0.0266]],

         [[-0.0272]],

         ...,

         [[-0.0385]],

         [[ 0.1080]],

         [[-1.3598]]],


        [[[ 2.9140]],

         [[-0.0687]],

         [[ 0.0144]],

         ...,

         [[ 0.0849]],

         [[-0.0033]],

         [[-3.0121]]],


        ...,


        [[[ 0.5411]],

         [[-0.0176]],

         [[ 0.0054]],

         ...,

         [[-0.0140]],

         [[-0.1117]],

         [[ 0.0722]]],


        [[[-0.1543]],

         [[ 0.0837]],

         [[ 0.0248]],

         ...,

         [[-0.0614]],

         [[ 0.0219]],

         [[-0.1945]]],


        [[[ 0.3079]],

         [[-0.0831]],

         [[-0.0359]],

         ...,

         [[ 0.1323]],

         [[ 0.0754]],

         [[-0.0676]]]])
1
tensor([[[[ 3.0491e-02]],

         [[ 5.1074e-03]],

tensor([[[[-1.4738e-02]],

         [[-4.5318e-02]],

         [[ 5.1411e-02]],

         ...,

         [[-9.1621e-02]],

         [[ 9.1919e-02]],

         [[ 8.0606e-02]]],


        [[[ 1.3920e+00]],

         [[ 2.2781e-03]],

         [[ 2.7065e-02]],

         ...,

         [[ 7.5357e-02]],

         [[ 4.2833e-02]],

         [[-1.3530e+00]]],


        [[[ 1.6945e-01]],

         [[-8.2329e-02]],

         [[-2.2501e-04]],

         ...,

         [[ 3.1431e-02]],

         [[-1.0314e-02]],

         [[-7.2344e-01]]],


        ...,


        [[[ 2.6639e-02]],

         [[-1.0998e-01]],

         [[-5.0307e-02]],

         ...,

         [[ 1.0653e-01]],

         [[ 8.8181e-02]],

         [[-7.5072e-01]]],


        [[[ 2.3465e+00]],

         [[-4.4923e-02]],

         [[-7.5777e-02]],

         ...,

         [[ 3.4890e-02]],

         [[ 1.5772e-01]],

         [[-3.1732e+00]]],


        [[[ 1.2180e-01]],

         [[-8.2696e-02]],

         [[-7.5657e-02]],

         

1
tensor([[[[-0.0362]],

         [[ 0.1201]],

         [[-0.0689]],

         ...,

         [[ 0.1705]],

         [[-0.0782]],

         [[-0.0568]]],


        [[[ 0.0465]],

         [[-0.0335]],

         [[ 0.0436]],

         ...,

         [[ 0.0255]],

         [[ 0.1516]],

         [[-0.0233]]],


        [[[-0.0738]],

         [[ 0.0391]],

         [[ 0.0099]],

         ...,

         [[ 0.0797]],

         [[ 0.0698]],

         [[-0.9769]]],


        ...,


        [[[-0.0352]],

         [[ 0.1134]],

         [[ 0.0120]],

         ...,

         [[-0.0050]],

         [[-0.0410]],

         [[-0.0497]]],


        [[[ 0.1170]],

         [[-0.0265]],

         [[-0.0219]],

         ...,

         [[-0.0109]],

         [[ 0.0409]],

         [[-0.1397]]],


        [[[-0.0242]],

         [[ 0.0714]],

         [[-0.0416]],

         ...,

         [[ 0.1555]],

         [[ 0.0548]],

         [[-0.0956]]]])
1
tensor([[[[-0.0924]],

         [[-0.0044]],

      

1
tensor([[[[ 0.1075]],

         [[ 0.0730]],

         [[-0.0161]],

         ...,

         [[ 0.0924]],

         [[ 0.0134]],

         [[ 0.0766]]],


        [[[ 0.0652]],

         [[ 0.0956]],

         [[-0.1461]],

         ...,

         [[-0.0275]],

         [[ 0.0855]],

         [[-0.1303]]],


        [[[ 1.8953]],

         [[ 0.0524]],

         [[-0.1203]],

         ...,

         [[-0.0569]],

         [[-0.0246]],

         [[-1.9590]]],


        ...,


        [[[ 1.7157]],

         [[-0.0652]],

         [[-0.1175]],

         ...,

         [[ 0.0366]],

         [[-0.0058]],

         [[-2.0164]]],


        [[[ 0.1150]],

         [[ 0.0500]],

         [[-0.1215]],

         ...,

         [[ 0.0569]],

         [[-0.0199]],

         [[-0.1607]]],


        [[[-0.0566]],

         [[-0.0672]],

         [[-0.0884]],

         ...,

         [[-0.1489]],

         [[ 0.0024]],

         [[-0.0989]]]])
1
tensor([[[[-0.0637]],

         [[-0.0054]],

      

1
tensor([[[[ 8.4396e-01]],

         [[-3.7945e-02]],

         [[-5.8732e-02]],

         ...,

         [[ 5.4352e-02]],

         [[ 2.3528e-02]],

         [[-8.4779e-01]]],


        [[[ 2.3541e-02]],

         [[ 1.4709e-03]],

         [[ 4.2565e-02]],

         ...,

         [[ 8.9452e-02]],

         [[ 4.9104e-02]],

         [[-2.3279e-02]]],


        [[[ 2.7006e+00]],

         [[-5.4697e-02]],

         [[ 8.4617e-02]],

         ...,

         [[-9.1881e-02]],

         [[-9.0941e-02]],

         [[-3.0429e+00]]],


        ...,


        [[[-8.1182e-02]],

         [[-1.2873e-02]],

         [[ 7.5274e-03]],

         ...,

         [[-4.9090e-03]],

         [[-4.4672e-02]],

         [[ 6.3056e-02]]],


        [[[ 2.2721e+00]],

         [[-1.1431e-01]],

         [[-1.1992e-02]],

         ...,

         [[ 5.7119e-02]],

         [[-1.2818e-01]],

         [[-1.5013e+00]]],


        [[[-9.0522e-02]],

         [[ 4.8661e-02]],

         [[-1.1959e-01]],

       

1
tensor([[[[ 1.3270e-03]],

         [[-1.5877e-02]],

         [[ 4.1008e-02]],

         ...,

         [[ 4.4434e-02]],

         [[-8.4363e-02]],

         [[ 1.0644e-02]]],


        [[[ 1.9969e+00]],

         [[ 2.7033e-02]],

         [[-4.0034e-02]],

         ...,

         [[ 2.5926e-03]],

         [[ 4.2239e-03]],

         [[-3.0722e+00]]],


        [[[ 2.3660e+00]],

         [[ 8.6235e-02]],

         [[ 1.2188e-01]],

         ...,

         [[ 9.1632e-02]],

         [[-4.4141e-02]],

         [[-1.8555e+00]]],


        ...,


        [[[ 1.2687e-01]],

         [[ 4.7416e-02]],

         [[-1.0032e-01]],

         ...,

         [[-8.4490e-02]],

         [[ 1.2855e-01]],

         [[-4.4328e-02]]],


        [[[ 5.0816e-01]],

         [[ 4.0887e-02]],

         [[ 5.5097e-02]],

         ...,

         [[-1.3221e-01]],

         [[-1.3313e-01]],

         [[-1.5250e-01]]],


        [[[ 1.7252e+00]],

         [[ 7.1883e-02]],

         [[ 5.9398e-02]],

       

1
tensor([[[[ 6.6874e-02]],

         [[-1.9033e-02]],

         [[-8.1656e-02]],

         ...,

         [[-1.5170e-02]],

         [[ 9.3593e-02]],

         [[-7.4272e-03]]],


        [[[ 3.2499e+00]],

         [[-3.2385e-02]],

         [[-3.0964e-02]],

         ...,

         [[-7.0646e-03]],

         [[-4.0303e-02]],

         [[-4.5286e+00]]],


        [[[ 3.3635e+00]],

         [[-4.6584e-02]],

         [[-6.5966e-02]],

         ...,

         [[ 4.1947e-02]],

         [[-6.5699e-02]],

         [[-3.1861e+00]]],


        ...,


        [[[ 7.5403e-02]],

         [[ 6.2283e-02]],

         [[-1.2048e-01]],

         ...,

         [[-9.6326e-02]],

         [[ 7.0605e-03]],

         [[-7.8741e-02]]],


        [[[ 5.3322e-01]],

         [[ 1.2993e-01]],

         [[-1.6460e-01]],

         ...,

         [[ 5.0827e-02]],

         [[-2.7680e-03]],

         [[-1.6433e+00]]],


        [[[-2.8775e-02]],

         [[ 1.4905e-02]],

         [[ 4.5654e-02]],

       

1
tensor([[[[-0.0112]],

         [[ 0.0755]],

         [[ 0.0387]],

         ...,

         [[-0.0697]],

         [[ 0.0052]],

         [[ 0.0287]]],


        [[[ 0.0686]],

         [[ 0.0480]],

         [[-0.0136]],

         ...,

         [[-0.0087]],

         [[-0.0468]],

         [[-0.0655]]],


        [[[ 0.0058]],

         [[-0.1616]],

         [[-0.0178]],

         ...,

         [[-0.1068]],

         [[-0.0032]],

         [[-0.0043]]],


        ...,


        [[[-0.0862]],

         [[ 0.0292]],

         [[-0.0049]],

         ...,

         [[-0.1063]],

         [[-0.1263]],

         [[-0.0217]]],


        [[[ 2.1967]],

         [[ 0.0482]],

         [[-0.0423]],

         ...,

         [[ 0.0126]],

         [[ 0.1064]],

         [[-2.6093]]],


        [[[ 0.9363]],

         [[-0.1169]],

         [[-0.0616]],

         ...,

         [[ 0.1443]],

         [[-0.0392]],

         [[-0.1659]]]])
1
tensor([[[[ 1.2011e+00]],

         [[ 7.2077e-02]],